# Impossible Chessboard Escape Puzzle

Solution implementation by Cory Leigh Rahman

Related links:

* Introduction video & walk-through: [The almost impossible chessboard puzzle](https://www.youtube.com/watch?v=as7Gkm7Y7h4) by Stand-up Maths
* Further discussion video: [The impossible chessboard puzzle](https://www.youtube.com/watch?v=wTJI_WuZSwE) by 3Blue1Brown
* Full breakdown website with interactive examples: [Impossible Escape?](https://datagenetics.com/blog/december12014/index.html) by DataGenetics

## Problem Prompt

Prisoner 1 walks in to a room, sees a chessboard where each square has a coin on top, flipped either to heads or tails.  The warden places the key under one of the squares, which prisoner 1 sees.  Before he leaves, he must turn over one and only one coin.  Prisoner 2 then walks in and is supposed to be able to figure out which squares the key is in just by looking at the arrangement of coins. The Prisoners can coordinate a plan ahead of time. What's the plan?


Here's how the scenario will go:
1. The warden will set up the board while the prisoners come up with a plan
2. Prisoner # 1 will witness which box the warden places the key under, then flip exactly one coin to try and convey this information, then leaves
3. Prisoner # 2 enters and has one chance to pick which box has the key under it

In [1]:
import numpy as np
import pandas as pd

## Set Up the Board

We need to make a representation of a chessboard with a randomly flipped coin in each square


In [2]:
def getChessboardOfCoins():
    """
    # Make the chessboard with randomized coins represented by 0 and 1
    # 1 = heads
    # 0 = tails
    """
    binary_8x8_grid = np.random.randint(0, 2, size=(8, 8))
    return pd.DataFrame(binary_8x8_grid)

chessboard = getChessboardOfCoins()
chessboard

,0,1,2,3,4,5,6,7
0,1,0,0,1,0,1,0,1
1,1,0,1,1,1,0,0,1
2,1,0,0,1,0,0,0,0
3,1,0,1,0,1,1,0,0
4,0,1,0,0,1,1,0,0
5,1,0,0,0,1,0,0,0
6,1,0,0,0,1,1,1,1
7,0,0,1,1,0,0,1,1


## The Prisoners Hatch a Plan

In [3]:
# The prisoners decide to give each square a unique ID
# Start with 0 and count left to right, then top to bottom
squares = np.zeros((8,8))
for i in range(8):
    squares[i] = np.arange(8*i, 8*(i+1))
chessboardSquareIDs = pd.DataFrame(squares).applymap(lambda x: int(x))

print('Each square on the chess board is given a unique ID 0 through 63:')

def getChessboardSquareIDs():
    squares = np.zeros((8,8))
    for i in range(8):
        squares[i] = np.arange(8*i, 8*(i+1))
    return pd.DataFrame(squares).applymap(lambda x: int(x))

chessboardSquareIDs = getChessboardSquareIDs()

Each square on the chess board is given a unique ID 0 through 63:


Because there are 64 squares each with their own unique ID (0 through 63), we need a minimum of 6 bits of information to represent any one square. For example:

* Square # 0 = 0 in binary
* Square # 63 = 111111 in binary
* And, for example, Square # 37 = 100101 in binary

Here is every square ID converted to binary:

In [4]:
def binaryString64ToInt(binaryString):
    return int(binaryString, 2)

def intToBinaryString64(num):
    return format(num, 'b').zfill(6)

chessboardBinaryUniqueSquareIDs = chessboardSquareIDs.applymap(intToBinaryString64)

print("Each square's unique ID represented in binary:")
chessboardBinaryUniqueSquareIDs

Each square's unique ID represented in binary:


,0,1,2,3,4,5,6,7
0,000000,000001,000010,000011,000100,000101,000110,000111
1,001000,001001,001010,001011,001100,001101,001110,001111
2,010000,010001,010010,010011,010100,010101,010110,010111
3,011000,011001,011010,011011,011100,011101,011110,011111
4,100000,100001,100010,100011,100100,100101,100110,100111
5,101000,101001,101010,101011,101100,101101,101110,101111
6,110000,110001,110010,110011,110100,110101,110110,110111
7,111000,111001,111010,111011,111100,111101,111110,111111


Now the challenge becomes how to precisely communicate 6 bits of information by flipping just one coin on the board. We have to be able to change any of the 6 digits simultaneously.

The board can be split up in 6 groups to allow any one coin change to have an effect on all, some, or none of the the groups. Each of the 6 groups will correspond to one of the digits of the binary ID we wish to convey. We can construct a binary string representing the position ID of the key by counting the number if heads-up coins in each section. In the binary string, even number of heads-up coins will mean 1, while an odd number of heads-up coins will mean 0.

![Sections](img/sections_datagenetics.png)
_Image Source: https://datagenetics.com/blog/december12014/index.html_

So, for example, if you flip Square ID 0 then the chess board's encoded number will not change at all. If you flip Square ID 1 then only the first digit of the binary will change. If you flip Square ID 5, then exactly the first and third digits of the binary will change, and none others. This system allows you to change the board's inherent randomly encoded binary to any number 0 to 63, allowing Prisoner # 2 to decode this message and discover the square containing the key.

In [5]:
class SectionDefinition:
    """
    We have defined 6 sections in such a way that allows one coin flip
    to have an effect on any combination of the 6 binary digits
    
    Sections:
    * 0 = Columns 1, 3, 5, 7
    * 1 = Columns 2, 3, 6, 7
    * 2 = Columns 4, 5, 6, 7
    * 3 = Rows 1, 3, 5, 7
    * 4 = Rows 2, 3, 6, 7
    * 5 = Rows 4, 5, 6, 7
    """
    
    def __init__(self):
        self._getSectionSwitcher = [
            self._getSection0, self._getSection1, self._getSection2,
            self._getSection3, self._getSection4, self._getSection5
        ]
    
    def get(self, sectionNumber, dataFrame, inSection = True):
        """
        Get a subsection of the provided chessboard based on the section number
        
        Parameters:
        * sectionNumber - the section
        * dataFrame - the chessboard
        * inSection (default:True) - set False to get outside the section number
        
        Example:
        `section1 = sections.get(1, chessboard)`
        """
        return self._getSectionSwitcher[sectionNumber](dataFrame, inSection)
    
    def _getSection0(self, dataFrame, inSection):
        return dataFrame.iloc[:,[1, 3, 5, 7]] if inSection else dataFrame.iloc[:,[0, 2, 4, 6]]
    def _getSection1(self, dataFrame, inSection):
        return dataFrame.iloc[:,[2, 3, 6, 7]] if inSection else dataFrame.iloc[:,[0, 1, 4, 5]]
    def _getSection2(self, dataFrame, inSection):
        return dataFrame.iloc[:,[4, 5, 6, 7]] if inSection else dataFrame.iloc[:,[0, 1, 2, 3]]
    def _getSection3(self, dataFrame, inSection):
        return dataFrame.iloc[[1, 3, 5, 7]] if inSection else dataFrame.iloc[[0, 2, 4, 6]]
    def _getSection4(self, dataFrame, inSection):
        return dataFrame.iloc[[2, 3, 6, 7]] if inSection else dataFrame.iloc[[0, 1, 4, 5]]
    def _getSection5(self, dataFrame, inSection):
        return dataFrame.iloc[[4, 5, 6, 7]] if inSection else dataFrame.iloc[[0, 1, 2, 3]]


sections = SectionDefinition()

print("For example, here is Section # 0 of the chessboard (columns 1, 3, 5, 7):")
sections.get(0, chessboard)

For example, here is Section # 0 of the chessboard (columns 1, 3, 5, 7):


,1,3,5,7
0,0,1,1,1
1,0,1,0,1
2,0,1,0,0
3,0,0,1,0
4,1,0,1,0
5,0,0,0,0
6,0,0,1,1
7,0,1,0,1


In [6]:
def oneIfEvenZeroIfOdd(num):
    if (num % 2) == 0:  
        return 1 
    else:  
        return 0

def countHeadsUpInSection(section):
    return section.to_numpy().sum()

def readBinaryLocationFromBoard(board, sections):
    digits = []
    for i in range(6):
        sectionOfBoard = sections.get(i, board)
        numberOfHeadsUpInSection = countHeadsUpInSection(sectionOfBoard)
        oneOrZero = oneIfEvenZeroIfOdd(numberOfHeadsUpInSection)
        digits.append(oneOrZero)
    binaryStringBasedOnSections = "".join([str(int) for int in digits])
    return binaryStringBasedOnSections

#TODO Maybe make this a class


## Prisoner # 1 is shown which box holds the key

In [7]:
# The warden places the key under one of the squares on the chess board

def getWardenKeyPlacement():
    keyLocationRow = np.random.randint(0, 8)
    keyLocationCol = np.random.randint(0, 8)
    return keyLocationRow * 8 + keyLocationCol

keyLocationSquareID = getWardenKeyPlacement()

print(f'The prison warden has placed the key under square # {keyLocationSquareID}')

The prison warden has placed the key under square # 8


Now that we have our target key location, Prisoner # 1 needs to decide which coin to flip to convey this information.

In [8]:
initialBinaryValue = readBinaryLocationFromBoard(chessboard, sections)
targetBinaryValue = intToBinaryString64(keyLocationSquareID)
print(f'Initial chessboard binary: {initialBinaryValue}')
print(f'Target chessboard binary:  {targetBinaryValue} (this represents "{keyLocationSquareID}", the location of the key)')

Initial chessboard binary: 000001
Target chessboard binary:  001000 (this represents "8", the location of the key)


Now we need to figure out which coin on the board will, when flipped, **turn the board's initial binary value into the target binary value**. This will allow Prisoner # 2 to read the read the location of the key from the board.

In [9]:
def getPositionOptionsForSection(sectionNumber, inSection):
    return sections.get(sectionNumber, chessboardSquareIDs, inSection).to_numpy().flatten()

def findWhichCoinToFlip(board, keyLocationID, sections):
    initialBinary = readBinaryLocationFromBoard(board, sections)
    targetBinary = intToBinaryString64(keyLocationID)

    # Get all positions with correct impact on a digit
    positionOptions = []
    for i in range(6):
        if initialBinary[i] != targetBinary[i]:
            positionOptions.append(getPositionOptionsForSection(i, True))
        else:
            positionOptions.append(getPositionOptionsForSection(i, False))
    
    positionsWithCorrectImpact = np.array(positionOptions).flatten()
    
    # Finds the most frequent position (should occur 6 times)
    coinToFlipSquareID = np.bincount(positionsWithCorrectImpact).argmax()
    
    #TODO Fix coinToFlipSquareID
    return coinToFlipSquareID

coinToFlipSquareID = findWhichCoinToFlip(chessboard, keyLocationSquareID, sections)

coinToFlipSquareID

36

In [10]:
def flipCoinForNewChessboardState(board, squareID):

    # Figure out where the target squareID is on the board
    rowLoc = int(squareID/8)
    colLoc = squareID % 8
    coin = board.iloc[rowLoc, colLoc]
    
#     print(f"rowLoc: {rowLoc}, colLoc: {colLoc}")
    
    # Flip the coin and return a new chessboard state
    newBoard = board.copy()
    if (coin == 1):
        # Coin is heads
        newBoard.iloc[rowLoc, colLoc] = 0
    else:
        # Coin is tails
        newBoard.iloc[rowLoc, colLoc] = 1
        
    return newBoard


newBoard = flipCoinForNewChessboardState(chessboard, coinToFlipSquareID)

newBinaryValue = readBinaryLocationFromBoard(newBoard, sections)

squareWithKeyChoice = binaryString64ToInt(newBinaryValue)

print()
print(f'Key Location Square ID: {keyLocationSquareID}')
print(f'Coin Flipped: {coinToFlipSquareID}')
print()
print(f'Initial chessboard binary: {initialBinaryValue}')
print(f'Target chessboard binary:  {targetBinaryValue} (this represents "{keyLocationSquareID}", the location of the key)')
print(f'New chessboard binary:     {newBinaryValue}')
print(f'Square with the key?: {squareWithKeyChoice} (Should be {keyLocationSquareID})')

if squareWithKeyChoice == keyLocationSquareID:
    print("Success!")


Key Location Square ID: 8
Coin Flipped: 36

Initial chessboard binary: 000001
Target chessboard binary:  001000 (this represents "8", the location of the key)
New chessboard binary:     001000
Square with the key?: 8 (Should be 8)
Success!


In [11]:
# Test the solution

def testPuzzleSolution(
        getChessboardOfCoins, getChessboardSquareIDs,
        binaryString64ToInt, intToBinaryString64,
        sections, readBinaryLocationFromBoard, getWardenKeyPlacement,
        findWhichCoinToFlip, flipCoinForNewChessboardState
    ):
    chessboard = getChessboardOfCoins()
    chessboardSquareIDs = getChessboardSquareIDs()
    initialBinaryStringValue = readBinaryLocationFromBoard(chessboard, sections)
    
    keyLocationSquareID = getWardenKeyPlacement()
    targetBinaryStringValue = intToBinaryString64(keyLocationSquareID)
    
    coinToFlipSquareID = findWhichCoinToFlip(chessboard, keyLocationSquareID, sections)
    newChessboard = flipCoinForNewChessboardState(chessboard, coinToFlipSquareID)
    newBinaryStringValue = readBinaryLocationFromBoard(newChessboard, sections)
    calculatedKeyLocationSquareID = binaryString64ToInt(newBinaryStringValue)
    
#     print(f'{calculatedKeyLocationSquareID} ({targetBinaryStringValue})')
#     print(f'{keyLocationSquareID} ({newBinaryStringValue})')
    return calculatedKeyLocationSquareID == keyLocationSquareID

def batchTestPuzzleSolution(count):
    success = 0
    failure = 0
    for i in range(count):
        result = testPuzzleSolution(
            getChessboardOfCoins, getChessboardSquareIDs,
            binaryString64ToInt, intToBinaryString64,
            sections, readBinaryLocationFromBoard, getWardenKeyPlacement,
            findWhichCoinToFlip, flipCoinForNewChessboardState
        )
        if (result):
            success += 1
        else:
            failure += 1
#     return {'success': success, 'failure': failure}
    return [success, failure]
    
success, failure = batchTestPuzzleSolution(100)
print(f"Result of {success + failure} random test runs:")
print(f"Success: {success}, failure: {failure}")


Result of 100 random test runs:
Success: 100, failure: 0
